## Importo librerías

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
!pip install geopandas

In [3]:
!pip install requests

## Google colab

Para ejecutar este notebook en Colab, primero descomenten y ejecuten las siguientes 3 celdas. Luego de ejecutar la notebook se reiniciará.

In [4]:
'''
# Montando datos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')
'''

"\n# Montando datos desde Google Drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [5]:
'''
import sys
sys.path.append('/aqui_reemplazar_path_drive/')
'''

"\nimport sys\nsys.path.append('/aqui_reemplazar_path_drive/')\n"

### Levanto ENMODO 2009
https://www.argentina.gob.ar/transporte/dgppse/publicaciones/encuestas

Esta notebook apunta directamente a datos subidos a github por lo cual no es necesario reemplazarla por data paths locales o en drive

In [6]:
hogares = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/hogares.csv')
personas = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/personas.csv')
viajes = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/viajes.csv')
etapas = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/etapas.csv')

/home/roliverio/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/roliverio/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Caracterización de los datos

Los datos utilizados en este cuaderno corresponden a los resultados de la Encuesta Origen-Destino de Buenos Aires (AMBA), Argentina del 2009. 
A partir de estos datos se calcularan indicadores en tres niveles: de Cantidad de Viajes, de Tiempo de Viajes, y de Distancia de Viajes.

Hay datos a nivel hogares, personas, viajes y etapas.

### Importando datos

In [7]:
import requests
import json
datastring = "https://raw.githubusercontent.com/RacoFernandez/enmodo/main/buenos-aires/source-geojson/departamentos-buenos_aires.json"
resp = requests.get(datastring)
departamentos = json.loads(resp.text)

In [8]:
datastring = "https://raw.githubusercontent.com/RacoFernandez/enmodo/main/buenos-aires/source-geojson/departamentos-ciudad_autonoma_de_buenos_aires.json"
resp = requests.get(datastring)
departamentos_caba = json.loads(resp.text)

In [9]:
import geopandas as gpd
departamentos_gpd = gpd.GeoDataFrame.from_features(departamentos["features"])

In [10]:
departamentos_gpd.departamento.unique()

array(['ALMIRANTE BROWN', 'LA PLATA', 'PRESIDENTE PERON',
       'CARLOS CASARES', 'SALADILLO', 'PEHUAJO', 'TRENQUE LAUQUEN',
       'GENERAL BELGRANO', 'LAS FLORES', 'GENERAL GUIDO', 'OLAVARRIA',
       'ROJAS', 'SAN ANTONIO DE ARECO', 'MAIPU', 'ARRECIFES',
       'CAPITAN SARMIENTO', 'CARMEN DE ARECO', 'PINAMAR', 'SALTO',
       'EXALTACION DE LA CRUZ', 'ESCOBAR', 'SAN MIGUEL',
       'SAN ANDRES DE GILES', 'JUNIN', 'MERCEDES', 'GENERAL VILLEGAS',
       'PILAR', 'LUJAN', 'HURLINGHAM', 'MORON', 'CHACABUCO',
       'MALVINAS ARGENTINAS', 'JOSE C PAZ', 'GENERAL SAN MARTIN',
       'MORENO', 'LINCOLN', 'ITUZAINGO', 'FLORENTINO AMEGHINO',
       'LA MATANZA', 'GENERAL RODRIGUEZ', 'SUIPACHA', 'TRES DE FEBRERO',
       'MERLO', 'CHIVILCOY', 'BRAGADO', 'LANUS', 'GENERAL LAS HERAS',
       'MARCOS PAZ', 'LOMAS DE ZAMORA', 'GENERAL VIAMONTE',
       'ESTEBAN ECHEVERRIA', 'EZEIZA', 'ALBERTI', 'FLORENCIO VARELA',
       'NAVARRO', 'CAÑUELAS', 'SAN VICENTE', 'LOBOS', 'BRANDSEN',
       'CARLOS T

## Limpieza de la capa de departamentos y join con datos de viajes de ENMODO (ya que la misma no viene con la data de la geometría de los viajes)

In [11]:
# Me quedo solo con algunos departamentos (los presentes en la ENMODO 2009)
departamentos_amba_gpd = departamentos_gpd[(departamentos_gpd['departamento'].str.contains("BROWN"))|
                  (departamentos_gpd['departamento'].str.contains("MATANZA"))|
                  (departamentos_gpd['departamento'].str.contains("LOPEZ"))|
                  (departamentos_gpd['departamento'].str.contains("ECHEVERRIA"))|
                  (departamentos_gpd['departamento'].str.contains("MORON"))|
                  (departamentos_gpd['departamento'].str.contains("ISIDRO"))|
                  (departamentos_gpd['departamento'].str.contains("PILAR"))|
                  (departamentos_gpd['departamento'].str.contains("LANUS"))|
                  (departamentos_gpd['departamento'].str.contains("JOSE C PAZ"))|
                  (departamentos_gpd['departamento'].str.contains("QUILMES"))|
                  (departamentos_gpd['departamento'].str.contains("HURLINGHAM"))|
                  (departamentos_gpd['departamento'].str.contains("SAN MARTIN"))|
                  (departamentos_gpd['departamento'].str.contains("AVELLANEDA"))|
                  (departamentos_gpd['departamento'].str.contains("LOMAS DE ZAMORA"))|
                  (departamentos_gpd['departamento'].str.contains("BERAZATEGUI"))|
                  (departamentos_gpd['departamento'].str.contains("ITUZAINGO"))|
                  (departamentos_gpd['departamento'].str.contains("ESCOBAR"))|
                  (departamentos_gpd['departamento'].str.contains("SAN MIGUEL"))|
                  (departamentos_gpd['departamento'].str.contains("MERLO"))|
                  (departamentos_gpd['departamento'].str.contains("FEBRERO"))|
                  (departamentos_gpd['departamento'].str.contains("SAN FERNANDO"))|
                  (departamentos_gpd['departamento'].str.contains("MORENO"))|
                  (departamentos_gpd['departamento'].str.contains("EZEIZA"))|
                  (departamentos_gpd['departamento'].str.contains("MALVINAS"))|
                  (departamentos_gpd['departamento'].str.contains("TIGRE"))|
                  (departamentos_gpd['departamento'].str.contains("PERON"))|
                  (departamentos_gpd['departamento'].str.contains("LA PLATA"))|
                  (departamentos_gpd['departamento'].str.contains("CAÑUELAS"))|
                  (departamentos_gpd['departamento'].str.contains("GENERAL LAS HERAS"))|
                  (departamentos_gpd['departamento'].str.contains("ZARATE"))|
                  (departamentos_gpd['departamento'].str.contains("MARCOS PAZ"))|
                  (departamentos_gpd['departamento'].str.contains("BRANDSEN"))|
                  (departamentos_gpd['departamento'].str.contains("ENSENADA"))|
                  (departamentos_gpd['departamento'].str.contains("MERCEDES"))|
                  (departamentos_gpd['departamento'].str.contains("BERISSO"))|
                  (departamentos_gpd['departamento'].str.contains("SAN VICENTE"))|
                  (departamentos_gpd['departamento'].str.contains("GENERAL RODRIGUEZ"))|
                  (departamentos_gpd['departamento'].str.contains("EXALTACION DE LA CRUZ"))|
                  (departamentos_gpd['departamento'].str.contains("FLORENCIO"))]

In [12]:
del(departamentos_gpd)

In [13]:
departamentos_caba_gpd = gpd.GeoDataFrame.from_features(departamentos_caba["features"])

In [14]:
del(departamentos_caba)

In [15]:
departamentos_amba_gpd = departamentos_amba_gpd.append(departamentos_caba_gpd)

In [16]:
viajes.partido = viajes.partido.str.upper()

In [17]:
viajes.TT_PARTIDO_ORIGEN = viajes.TT_PARTIDO_ORIGEN.str.upper()

In [18]:
viajes.TT_PARTIDO_DESTINO = viajes.TT_PARTIDO_DESTINO.str.upper()

In [19]:
cols = viajes.select_dtypes(include=[np.object]).columns
viajes[cols] = viajes[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

/tmp/ipykernel_928688/1910569586.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = viajes.select_dtypes(include=[np.object]).columns


In [20]:
dict = {"ALTE. BROWN" : 'ALMIRANTE BROWN', "E. ECHEVERRIA": 'ESTEBAN ECHEVERRIA', "JOSE C. PAZ": 'JOSE C PAZ',
         "PTE. PERON": 'PRESIDENTE PERON', "CORONEL BRANDSEN": 'BRANDSEN', "CANUELAS": 'CAÑUELAS', "SOLDATI": 'VILLA SOLDATI',
        "NUNEZ": 'NUÑEZ'}
viajes=viajes.replace({"partido": dict, "TT_PARTIDO_ORIGEN": dict, "TT_PARTIDO_DESTINO": dict})

In [21]:
barrios_caba = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.csv')

In [22]:
viajes = viajes.merge(barrios_caba, left_on='TT_LOCALIDAD_ORIGEN', right_on='barrio', how='left')

In [23]:
viajes.pop("WKT")
viajes.pop("barrio")
viajes.pop("perimetro")
viajes.pop("area")

0                  NaN
1         2.289008e+06
2         2.043711e+06
3         4.342280e+06
4                  NaN
              ...     
102779             NaN
102780    7.398179e+06
102781    7.398179e+06
102782    8.590784e+06
102783    2.289008e+06
Name: area, Length: 102784, dtype: float64

In [24]:
viajes = viajes.merge(barrios_caba, left_on='TT_LOCALIDAD_DESTINO', right_on='barrio', how='left')

In [25]:
viajes.pop("WKT")
viajes.pop("barrio")
viajes.pop("perimetro")
viajes.pop("area")

0         2.289008e+06
1         2.043711e+06
2         4.342280e+06
3                  NaN
4                  NaN
              ...     
102779             NaN
102780    2.289008e+06
102781    2.198622e+06
102782             NaN
102783             NaN
Name: area, Length: 102784, dtype: float64

In [26]:
viajes = viajes.rename(columns={'comuna_x': 'comuna_origen', 'comuna_y': 'comuna_destino'})

In [27]:
viajes.comuna_origen.fillna(viajes.TT_PARTIDO_ORIGEN, inplace = True)

In [28]:
viajes.comuna_destino.fillna(viajes.TT_PARTIDO_DESTINO, inplace = True)

In [29]:
viajes.comuna_origen = viajes.comuna_origen.astype(str)

In [30]:
viajes.comuna_destino = viajes.comuna_destino.astype(str)

In [31]:
viajes.comuna_origen.unique()

array(['LA MATANZA', '1.0', '3.0', 'ALMIRANTE BROWN', '2.0',
       'VICENTE LOPEZ', 'SAN ISIDRO', 'CAMPANA', 'TIGRE',
       'LOMAS DE ZAMORA', 'MORON', '4.0', '13.0', 'SAN FERNANDO', '14.0',
       '6.0', '15.0', 'MORENO', '12.0', 'GENERAL SAN MARTIN', '11.0',
       'LANUS', 'EZEIZA', '9.0', 'MERLO', '5.0', '10.0',
       'ESTEBAN ECHEVERRIA', '7.0', 'LA PLATA', 'CAP. FED. DE 19', '8.0',
       'ITUZAINGO', 'TRES DE FEBRERO', 'PILAR', 'JOSE C PAZ',
       'SAN MIGUEL', 'AVELLANEDA', 'GENERAL RODRIGUEZ', 'QUILMES',
       'FLORENCIO VARELA', 'HURLINGHAM', 'CAP. FED. DE 21',
       'FUERA DE AREA - NORTE', 'BERAZATEGUI', 'ESCOBAR', 'LUJAN',
       'MALVINAS ARGENTINAS', 'SIN DATO', 'FUERA DE AREA - SUR',
       'SAN VICENTE', 'CAÑUELAS', 'GENERAL LAS HERAS', 'PRESIDENTE PERON',
       'ZARATE', 'MARCOS PAZ', 'BRANDSEN', 'CAP. FED. DE 10', 'ENSENADA',
       'EXALTACION DE LA CRUZ', 'BERISSO', 'MERCEDES'], dtype=object)

In [32]:
dict = {"1.0" : 'COMUNA 1', "2.0" : 'COMUNA 2', "3.0" : 'COMUNA 3', "4.0" : 'COMUNA 4',
        "5.0" : 'COMUNA 5', "6.0" : 'COMUNA 6', "7.0" : 'COMUNA 7', "8.0" : 'COMUNA 8',
        "9.0" : 'COMUNA 9', "10.0" : 'COMUNA 10', "11.0" : 'COMUNA 11', "12.0" : 'COMUNA 12',
        "13.0" : 'COMUNA 13', "14.0" : 'COMUNA 14', "15.0" : 'COMUNA 15'}
viajes=viajes.replace({"comuna_origen": dict, "comuna_destino": dict})

In [33]:
viajes_gpd = departamentos_amba_gpd.merge(viajes, left_on = 'departamento', right_on='comuna_origen', how='right')

In [34]:
viajes_gpd = viajes_gpd.merge(departamentos_amba_gpd, left_on = 'comuna_destino', right_on = 'departamento', how='left')

In [35]:
viajes_gpd = viajes_gpd.rename(columns={'geometry_x': 'geometry_origen', 'geometry_y': 'geometry_destino'})

In [36]:
viajes_gpd.pop("id_x")
viajes_gpd.pop("departamento_x")
viajes_gpd.pop("cabecera_x")
viajes_gpd.pop("provincia_x")
viajes_gpd.pop("id_y")
viajes_gpd.pop("departamento_y")
viajes_gpd.pop("cabecera_y")
viajes_gpd.pop("provincia_y")

0         CIUDAD AUTONOMA DE BUENOS AIRES
1         CIUDAD AUTONOMA DE BUENOS AIRES
2         CIUDAD AUTONOMA DE BUENOS AIRES
3                            BUENOS AIRES
4                            BUENOS AIRES
                       ...               
102779                       BUENOS AIRES
102780    CIUDAD AUTONOMA DE BUENOS AIRES
102781    CIUDAD AUTONOMA DE BUENOS AIRES
102782                       BUENOS AIRES
102783                       BUENOS AIRES
Name: provincia_y, Length: 102784, dtype: object

In [37]:
viajes_gpd = viajes_gpd.set_geometry("geometry_origen")

In [38]:
viajes_gpd['origen_x_coord'] = viajes_gpd['geometry_origen'].centroid.x
viajes_gpd['origen_y_coord'] = viajes_gpd['geometry_origen'].centroid.y

In [39]:
viajes_gpd = viajes_gpd.set_geometry("geometry_destino")

In [40]:
viajes_gpd['destino_x_coord'] = viajes_gpd['geometry_destino'].centroid.x
viajes_gpd['destino_y_coord'] = viajes_gpd['geometry_destino'].centroid.y

In [41]:
# filtro viajes sin origen o destino especificado
viajes_gpd = viajes_gpd[(viajes_gpd.comuna_origen != 'FUERA DE AREA - NORTE')&(viajes_gpd.comuna_origen != 'FUERA DE AREA - SUR')&(viajes_gpd.comuna_origen != 'SIN DATO')&
            (viajes_gpd.comuna_destino != 'FUERA DE AREA - NORTE')&(viajes_gpd.comuna_destino != 'FUERA DE AREA - SUR')&(viajes_gpd.comuna_destino != 'SIN DATO')]

## Modo

In [42]:
etapas['Modo'] = etapas['Modo'].replace(['Ferrocarril', 'Subte / Premetro', 'A pie', 'Automovil / conductor', 'Taxi', 'Automovil / acompañante',
       'Colectivo', 'Bicicleta', 'Remis', 'Charter / combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'],
        ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta', 'Remis', 'Charter/combi',
       'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'])

In [43]:
# Creo sub-categorías de agrupación en Etapas

etapas_cats_old = ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta',
               'Remis', 'Charter/combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva']
etapas_cats_new = ['Público', 'Público', 'No Motorizado', 'Auto/Moto', 'A demanda', 'Auto/Moto', 'Público', 'No Motorizado',
               'A demanda', 'A demanda', 'A demanda', 'Auto/Moto', 'A demanda', 'A demanda', 'A demanda']


etapas['Modo_agg'] = etapas['Modo'].replace(etapas_cats_old, etapas_cats_new)

# Creo variables dummies para agregación de etapas a nivel de viaje
etapas = pd.concat([etapas, pd.get_dummies(etapas.Modo)], axis=1)
etapas = pd.concat([etapas, pd.get_dummies(etapas.Modo_agg)], axis=1)

In [44]:
particion_etapas = etapas.groupby(['Modo', 'Modo_agg']).wt1.sum().round().astype(int).reset_index()
particion_etapas.columns = ['Modo', 'Modo_agg', 'Etapas']
particion_etapas['Etapas_porc'] = (particion_etapas.Etapas / particion_etapas.Etapas.sum() * 100).round(1)

### Ahora en tabla viajes

In [45]:
etapas.columns

Index(['Unnamed: 0', 'fraccion', 'partido', 'LOCALIDA', 'ETAPANUM', 'MOTIVOCA',
       'ESTENT', 'ESTSAL', 'TARIFABO', 'TIPOTARI', 'TIPOGARA', 'TIPOCOST',
       'COSTOPEA', 'COSTOTAX', 'CUACAANT', 'CUACADES', 'Modo', 'COSTOEST',
       'ESTACENT', 'ESTACSAL', 'LINEAENT', 'LINEASAL', 'ESTACSUB', 'ESTACBAJ',
       'ESPEHORA', 'ESPEMINU', 'DURAHORA', 'DURAMINU', 'TAXICOM',
       'Z_TXTLINEA_NUMERICO', 'Z_TXTLINEA_SIN_CODIGO', 'wt1', 'z_tarifabo2',
       'z_frecuenc_imput', 'z_costota2_2', 'z_costopea', 'z_COSTOEST', 'IDH',
       'IDP', 'IDV', 'IDE', 'Modo_agg', 'A pie', 'Automovil/acompañante',
       'Automovil/conductor', 'Bicicleta', 'Bus empresa', 'Charter/combi',
       'Colectivo', 'Ferrocarril', 'Lancha Colectiva', 'Motocicleta', 'Otros',
       'Remis', 'Subte/Premetro', 'Taxi', 'Transporte escolar', 'A demanda',
       'Auto/Moto', 'No Motorizado', 'Público'],
      dtype='object')

In [46]:
viajes_gpd_etapa = viajes_gpd.merge(etapas, on = 'IDV', how='left')

In [47]:
viajes_gpd_etapa = viajes_gpd_etapa.merge(personas, left_on = 'IDP_x', right_on = 'IDP', how='left')

In [48]:
viajes_gpd = viajes_gpd.merge(personas, on = 'IDP', how='left')

In [49]:
'''
# Otra alternativa podría ser:
Viajes_agg = etapas.groupby(['IDH', 'IDP', 'IDV']).agg({'Modo':'max', 'modo_agg':'max', 'Ferrocarril':'sum', 'Subte/Premetro':'sum', 'A pie':'sum', 'Automovil/conductor':'sum', 'Taxi':'sum', 
                                                  'Automovil/acompañante':'sum', 'Colectivo':'sum', 'Bicicleta':'sum', 'Remis':'sum', 'Charter/combi':'sum', 
                                                  'Bus empresa':'sum', 'Motocicleta':'sum', 'Otros':'sum', 'Transporte escolar':'sum', 'Lancha Colectiva':'sum', 
                                                  'Público':'sum', 'No Motorizado':'sum', 'Auto/Moto':'sum', 'A demanda':'sum', 'wt1':'max'}).reset_index()

Viajes_agg['Cant_etapas'] = Viajes_agg[['Público', 'No Motorizado', 'Auto/Moto', 'A demanda']].sum(axis=1)
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&(Viajes_agg.Público>0), 'Modo'] = 'Multietapa'
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&((Viajes_agg.Modo!='Multietapa')), 'Modo'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Otros Multietapa', 'Modo_agg'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Multietapa', 'Modo_agg'] = 'Multietapa'

Viajes_agg['Modo_agg2'] = Viajes_agg.Modo_agg.replace(['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'Otros Multietapa', 'Multietapa'], ['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'A demanda', 'Público'])
'''

"\n# Otra alternativa podría ser:\nViajes_agg = etapas.groupby(['IDH', 'IDP', 'IDV']).agg({'Modo':'max', 'modo_agg':'max', 'Ferrocarril':'sum', 'Subte/Premetro':'sum', 'A pie':'sum', 'Automovil/conductor':'sum', 'Taxi':'sum', \n                                                  'Automovil/acompañante':'sum', 'Colectivo':'sum', 'Bicicleta':'sum', 'Remis':'sum', 'Charter/combi':'sum', \n                                                  'Bus empresa':'sum', 'Motocicleta':'sum', 'Otros':'sum', 'Transporte escolar':'sum', 'Lancha Colectiva':'sum', \n                                                  'Público':'sum', 'No Motorizado':'sum', 'Auto/Moto':'sum', 'A demanda':'sum', 'wt1':'max'}).reset_index()\n\nViajes_agg['Cant_etapas'] = Viajes_agg[['Público', 'No Motorizado', 'Auto/Moto', 'A demanda']].sum(axis=1)\nViajes_agg.loc[(Viajes_agg.Cant_etapas>1)&(Viajes_agg.Público>0), 'Modo'] = 'Multietapa'\nViajes_agg.loc[(Viajes_agg.Cant_etapas>1)&((Viajes_agg.Modo!='Multietapa')), 'Modo'] = 'Otros

## Cálculo distancia

In [50]:
def manhattan_distance(a, b):
    return sum(abs(e1-e2) for e1, e2 in zip(a,b)) * 100

In [51]:
def distance_calc (row):
    start = (row['origen_y_coord'], row['origen_x_coord'])
    stop = (row['destino_y_coord'], row['destino_x_coord'])

    return manhattan_distance(start, stop)

In [52]:
viajes_gpd['distance'] = viajes_gpd.apply (lambda row: distance_calc (row),axis=1)

In [53]:
viajes_gpd_etapa['distance'] = viajes_gpd_etapa.apply (lambda row: distance_calc (row),axis=1)

## Cálculo de tiempo

In [54]:
def time_difference(row):
  time_diff = (row['HORALLEG'] - row['HORASALI']) * 60 + (row['MINLLEGA'] - row['MINSALID'])
  return(time_diff)

In [55]:
viajes_gpd['time_diff'] = viajes_gpd.apply (lambda row: time_difference (row),axis=1)

In [56]:
viajes_gpd_etapa['time_diff'] = viajes_gpd_etapa.apply (lambda row: time_difference (row),axis=1)

## Horario punta

In [57]:
def tipo_horario(row):
    if (row['HORASALI'] >= 7.0 and row['HORASALI'] < 11) or (row['HORASALI'] >= 16 and row['HORASALI'] < 20) :
        return 'punta'
    else:
        return 'valle'

In [58]:
viajes_gpd['tipo_horario'] = viajes_gpd.apply(tipo_horario, axis=1)

In [59]:
viajes_gpd_etapa['tipo_horario'] = viajes_gpd_etapa.apply(tipo_horario, axis=1)

## Horario punta mañana

In [60]:
def tipo_horario_punta_manana(row):
    if (row['HORASALI'] >= 7.0 and row['HORASALI'] < 11) :
        return True
    else:
        return False

In [61]:
viajes_gpd['punta_manana'] = viajes_gpd.apply(tipo_horario_punta_manana, axis=1)

In [62]:
viajes_gpd_etapa['punta_manana'] = viajes_gpd_etapa.apply(tipo_horario_punta_manana, axis=1)

## Recategorizo modo para el posterior calculo de indicadores

In [63]:
viajes_gpd_etapa['Modo_indicador'] = viajes_gpd_etapa['Modo']

In [64]:
dict = {"Automovil/acompañante" : 'Auto', "Automovil/conductor": 'Auto', "Bus empresa": 'Otros',
         "Charter/combi": 'Otros', "Colectivo": 'Público', "Ferrocarril": 'Público', "Lancha Colectiva": 'Público',
        "Remis": 'Otros', "Subte/Premetro": 'Público', "Taxi": 'Otros', "Transporte escolar": 'Otros'}

viajes_gpd_etapa=viajes_gpd_etapa.replace({"Modo_indicador": dict})

In [65]:
# Cambio valores lugar de trabajo
viajes_gpd['ACTIORIG_AGG'] = viajes_gpd['ACTIORIG'] 
dict = {'Asunto laboral': 'Trabajo', 'Lugar de trabajo': 'Trabajo'}
viajes_gpd=viajes_gpd.replace({"ACTIORIG_AGG": dict})

## Join de Viajes - Etapa + Hogares

In [66]:
viajes_gpd_etapa_hogares = viajes_gpd_etapa.merge(hogares, on = 'IDH', how='left')

/home/roliverio/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x', 'fraccion_x', 'LOCALIDA_x', 'wt1_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [67]:
viajes_gpd_hogares = viajes_gpd.merge(hogares, left_on = 'IDH_x', right_on = 'IDH', how='left')

## Generación de csv

In [68]:
viajes_gpd_hogares.shape

(102560, 96)

In [69]:
viajes_gpd_etapa_hogares.shape

(117535, 156)

In [70]:
viajes_gpd_hogares_to_save = viajes_gpd_hogares.drop(['geometry_origen', 'geometry_destino'], axis=1)


In [71]:
viajes_gpd_hogares_to_save = pd.DataFrame(viajes_gpd_hogares_to_save)

In [72]:
# Cambiar por path propio de drive o local
data_path = '/home/roliverio/Documentos/ADDI/enmodo/buenos-aires/'

In [73]:
viajes_gpd_hogares_to_save.to_csv(data_path + 'output-csv/viajes_gpd_hogares.csv', index=False)

In [74]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares.drop(['geometry_origen', 'geometry_destino'], axis=1)

In [75]:
viajes_gpd_etapa_hogares_to_save = pd.DataFrame(viajes_gpd_etapa_hogares_to_save)

In [76]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares_to_save.loc[:,~viajes_gpd_etapa_hogares_to_save.columns.duplicated()]

In [77]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares_to_save.drop(['Unnamed: 0_x', 'HORAINI', 'HORAFIN', 'wt1_x', 'z_frecuenc_imput_x', 'Unnamed: 0_y', 'wt1_y', 'IDH_y',
                                       'IDP_y', 'Unnamed: 0_x', 'wt1_x', 'CDNINGUN', 'CDMOTORA', 'CDMENTAL', 'CDVISUAL', 'CDAUDITI', 'CDHABLA', 'CDOTRAS', 'z_cocherasino',
                                       'z_tieneciclomo', 'z_tienemoto', 'z_tieneauto', 'z_gastoco2', 'z_gastoma2', 'A pie', 'Automovil/acompañante', 'Automovil/conductor',
                                       'Bicicleta', 'Bus empresa', 'Charter/combi', 'Colectivo', 'Ferrocarril', 'Lancha Colectiva', 'Motocicleta', 'Otros', 'Remis', 'Subte/Premetro', 'Taxi', 'Transporte escolar', 'A demanda', 
                                       'Auto/Moto', 'No Motorizado', 'TIPOGARA', 'TIPOCOST', 'COSTOPEA', 'COSTOTAX', 'CUACAANT', 'CUACADES', 'ESPEHORA', 'ESPEMINU', 'DURAHORA', 'DURAMINU',
                                       'IDH_x', 'IDP_x', 'origen_x_coord', 'origen_y_coord', 'destino_x_coord', 'destino_y_coord'], axis=1)

In [78]:
viajes_gpd_hogares_to_save.shape

(102560, 94)

In [79]:
viajes_gpd_etapa_hogares_to_save.shape

(117535, 90)

In [80]:
viajes_gpd_etapa_hogares_to_save.to_csv(data_path + 'output-csv/viajes_gpd_etapa_hogares.csv', index=False)

## Generación de geojson

### viajes

In [81]:
origenes_viajes = pd.DataFrame(viajes_gpd_hogares)
destinos_viajes = pd.DataFrame(viajes_gpd_hogares)

In [82]:
origenes_viajes = gpd.GeoDataFrame(origenes_viajes, geometry='geometry_origen')
destinos_viajes = gpd.GeoDataFrame(destinos_viajes, geometry='geometry_destino')

In [83]:
origenes_viajes.geometry_origen = origenes_viajes.geometry_origen.centroid
destinos_viajes.geometry_destino = destinos_viajes.geometry_destino.centroid

In [84]:
origenes_viajes = origenes_viajes.loc[:,~origenes_viajes.columns.duplicated()]
destinos_viajes = destinos_viajes.loc[:,~destinos_viajes.columns.duplicated()]

In [85]:
origenes_viajes.drop(['geometry_destino'], axis=1, inplace = True)
destinos_viajes.drop(['geometry_origen'], axis=1, inplace = True)

In [86]:
origenes_viajes.to_file(data_path + 'output-geojson/origenes_viajes.geojson', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'output-geojson/destinos_viajes.geojson', driver='GeoJSON')

### viajes_etapa

In [87]:
origenes_viajes_etapa = pd.DataFrame(viajes_gpd_etapa_hogares)
destinos_viajes_etapa = pd.DataFrame(viajes_gpd_etapa_hogares)

In [88]:
origenes_viajes_etapa = gpd.GeoDataFrame(origenes_viajes_etapa, geometry='geometry_origen')
destinos_viajes_etapa = gpd.GeoDataFrame(destinos_viajes_etapa, geometry='geometry_destino')

In [89]:
origenes_viajes_etapa.geometry_origen = origenes_viajes_etapa.geometry_origen.centroid
destinos_viajes_etapa.geometry_destino = destinos_viajes_etapa.geometry_destino.centroid

In [90]:
origenes_viajes_etapa = origenes_viajes_etapa.loc[:,~origenes_viajes_etapa.columns.duplicated()]
destinos_viajes_etapa = destinos_viajes_etapa.loc[:,~destinos_viajes_etapa.columns.duplicated()]

In [91]:
origenes_viajes_etapa.drop(['geometry_destino'], axis=1, inplace = True)
destinos_viajes_etapa.drop(['geometry_origen'], axis=1, inplace = True)

In [ ]:
origenes_viajes_etapa.to_file(data_path + 'output-geojson/origenes_viajes_etapa.geojson', driver='GeoJSON')
destinos_viajes_etapa.to_file(data_path + 'output-geojson/destinos_viajes_etapa.geojson', driver='GeoJSON')

## Generación de shapefile

### viajes

In [ ]:
origenes_viajes.to_file(data_path + 'output-shp/origenes_viajes.shp', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'output-shp/destinos_viajes.shp', driver='GeoJSON')

### viajes_etapa

In [ ]:
origenes_viajes_etapa.to_file(data_path + 'output-shp/origenes_viajes_etapa.shp', driver='GeoJSON')
destinos_viajes_etapa.to_file(data_path + 'output-shp/destinos_viajes_etapa.shp', driver='GeoJSON')

## Cálculo de indicadores

#### Distancia de viajes en automóvil privado + Distancia de viajes en transporte público

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('distance')['distance']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('distance')['distance']

#### Distancia de viajes por motivo estudio + Distancia de viajes por motivo trabajo

In [ ]:
viajes_gpd.groupby(['ACTIORIG_AGG']).mean('distance')['distance']

In [ ]:
 viajes_gpd.groupby(['ACTIORIG_AGG', 'SEXO']).mean('distance')['distance']

#### Proporción de viajes intrazonales versus interzonales

In [ ]:
viajes_gpd["es_caba_origen"] = viajes_gpd["comuna_origen"].str.contains("COMUNA")

In [ ]:
viajes_gpd["es_caba_destino"] = viajes_gpd["comuna_destino"].str.contains("COMUNA")

In [ ]:
viajes_gpd.loc[((viajes_gpd['es_caba_origen'] == True) & (viajes_gpd['es_caba_destino'] == True)) |
               ((viajes_gpd['es_caba_origen'] == False) & (viajes_gpd['es_caba_destino'] == False)), 'tipo_viaje_zona'] = 'Intrazonal'  

In [ ]:
viajes_gpd.loc[((viajes_gpd['es_caba_origen'] == True) & (viajes_gpd['es_caba_destino'] == False)) |
               ((viajes_gpd['es_caba_origen'] == False) & (viajes_gpd['es_caba_destino'] == True)), 'tipo_viaje_zona'] = 'Interzonal'  

In [ ]:
viajes_gpd.groupby('tipo_viaje_zona').size()

In [ ]:
viajes_gpd.shape

#### Tiempo promedio de viaje al trabajo en transporte público


In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo promedio de viaje en hora punta de mañana

In [ ]:
viajes_gpd.groupby(['tipo_horario']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd.groupby(['tipo_horario', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo de viaje en transporte público en hora punta en la mañana


In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'punta_manana']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'punta_manana', 'SEXO']).mean('time_diff')['time_diff']

#### Brecha de tiempo entre hora punta y valle del transporte público

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'tipo_horario']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'tipo_horario', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo promedio de viaje por modo y tipo de transporte

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_indicador']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.time_diff.mean()

#### Distribución y Participación Modal de los viajes por clasificador socioeconómico


In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=True)

In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=False)

Proporción entre los viajes en transporte público y privado por clasificador socioeconómico


In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo_indicador')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=True)

In [ ]:
viajes_gpd_etapa.groupby('Modo_indicador')['ORDEN_y'].value_counts(normalize=True)

Tasa promedio de viajes diarios en transporte privado por clasificador económico

Cantidad de pasajeros por modo (demanda) por cada cien mil habitantes

Porcentaje de la población dentro del área de influencia de la red de transporte público


Tasa de buses de transporte público por cada cien mil habitantes

Cantidad de vehículos de transporte privado

Cantidad de vehículos del transporte público impulsados por energías no fósiles (eléctricos)

Tasa de muertes en accidentes de tráfico

Peso monetario del transporte público en el presupuesto familiar

Brecha entre hombres y mujeres de viajes en la noche respecto al total de viajes


Relación combustible y costo ticket transporte público


## Coeficiente de movilidad (accesibilidad e interacción) y de inmovilidad

In [ ]:
!pip3 install seaborn

In [ ]:
import seaborn as sns
from scipy import stats
from scipy import sparse
import scipy.sparse.linalg.dsolve as linsolve
from sympy import *

In [ ]:
def generate_groups(df):
    # Generating interception points
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['n_viajes'],df['tiempo_total']) # linear equation
    median = df.tiempo_total.median() # time median
    max_value_x= df.n_viajes.max() # max trip number
    min_value_x= df.n_viajes.min() # max trip number
    max_value_y= df.tiempo_total.max() # max time
    min_value_y= df.tiempo_total.min() # max time
    interception = linsolve([slope*x+intercept-y, median-y ], (x, y))
    (x0, y0) = next(iter(interception)) # interception median time and trip number
    
    #interception points
    x_points=[]
    y_points=[]
    for x_point in range(1,max_value_x+1):
        interception_2 = linsolve([slope*x+intercept-y, x_point-x ], (x, y))
        (x1, y1) = next(iter(interception_2)) # interception points
        x_points.append(x1)
        y_points.append(y1)
    if x0 > 0:    
        #B2
        print('Generando grupo B2...')
        b2 = df[(df.tiempo_total >= y0) & (df.n_viajes >= x0)].assign(group='b2')

        #B1
        print('Generando grupo B1...')
        b1 = pd.DataFrame()
        for i in range(int(x0)+1,len(x_points)):
            temp = df[(df.tiempo_total < median) & (df.tiempo_total>=y_points[i-1]) & (df.n_viajes>=x_points[i-1]) & (df.n_viajes< x_points[i])]
            b1 = pd.concat([b1, temp], axis=0)
        b1 = b1.assign(group='b1')

        #A1
        print('Generando grupo A1...')
        a1 = pd.DataFrame()
        for i in range(int(x0)+1,len(x_points)):
            temp = df[(df.tiempo_total<y_points[i-1]) & (df.n_viajes>=x_points[i-1]) & (df.n_viajes< x_points[i])]
            a1 = pd.concat([a1, temp], axis=0)
        a1 = a1.assign(group='a1')

        #A2
        print('Generando grupo A2...')
        a2 = df[(df.n_viajes > 1) & (df.n_viajes < x0) & (df.tiempo_total < y0) & (df.tiempo_total >= 0)].assign(group='a2')

        #A3
        print('Generando grupo A3...')
        a3 = pd.DataFrame()
        for i in range(1,int(x0)+1):
            temp = df[(df.tiempo_total >= median) & (df.tiempo_total<y_points[i-1]) & (df.n_viajes>1) & (df.n_viajes>=x_points[i-1]) & (df.n_viajes< x_points[i])]
            a3 = pd.concat([a3, temp], axis=0)
        a3 = a3.assign(group='a3')

        #B3
        print('Generando grupo B3...')
        b3 = pd.DataFrame()
        for i in range(1,int(x0)+1):
            temp = df[(df.tiempo_total>=y_points[i-1]) & (df.n_viajes>1) & (df.n_viajes>=x_points[i-1]) & (df.n_viajes< x_points[i])]
            b3 = pd.concat([b3, temp], axis=0)
        b3 = b3.assign(group='b3')

        #C
        print('Generando grupo C...')
        c = df[(df.tiempo_total < y_points[0]) & (df.n_viajes == 1)].assign(group='c')

        #D
        print('Generando grupo D...')
        d = df[(df.tiempo_total >= y_points[0]) & (df.n_viajes == 1)].assign(group='d')

        return pd.concat([b2,b1,a1,a2,a3,b3,c,d], axis=0)

In [ ]:
def calculate_size_1p(df, group_n, fe):
    return df[df['group']==group_n][fe].sum()

In [ ]:
def calculate_indicators(df, fe):
    pop_total = df[fe].sum()
    print('Tamaño población: {}'.format(pop_total))
    m = (calculate_size_1p(df, 'a1', fe) + calculate_size_1p(df, 'a2', fe) + calculate_size_1p(df, 'a3', fe)) / pop_total
    m1 = (calculate_size_1p(df, 'a1', fe)) / pop_total
    m2 = (calculate_size_1p(df, 'a2', fe) + calculate_size_1p(df, 'a3', fe)) / pop_total
    m3 = (calculate_size_1p(df, 'b3', fe)) / pop_total
    m4 = (calculate_size_1p(df, 'b1', fe)) / pop_total
    m5 = (calculate_size_1p(df, 'b2', fe)) / pop_total
    m6 = (calculate_size_1p(df, 'c', fe)) / pop_total
    m7 = (calculate_size_1p(df, 'd', fe)) / pop_total
    m8 = (calculate_size_1p(df, 'c', fe) + calculate_size_1p(df, 'd', fe)) / pop_total
    m9 = df[df.tiempo_total <= 15][fe].sum() / pop_total
    m10 = df[df.n_viajes == 2][fe].sum() / pop_total
    
    data = {'Medida': ['(A1+A2+A3)/P', 'A1/P', '(A2+A3)/P', 'B3/P',
                       'B1/P', 'B2/P', 'C/P', 'D/P', '(C+D)/P', '<=15 min', '2 viajes diarios'],
           'Valor':[m, m1,m2,m3,m4,m5,m6,m7,m8,m9,m10]}
    
    return pd.DataFrame(data)   

In [ ]:
def plot_lmplot(df, hue=None, col=None, col_wrap=None, col_order=None,filename=None):
    
    ax = sns.lmplot(data=df, x="n_viajes", y="tiempo_total", hue=hue, 
                    col=col, col_wrap=col_wrap, col_order=col_order, 
                    aspect=1.5, height=4,  line_kws={'color': 'blue'},
                    scatter_kws={'color': 'grey', 'alpha':0.2}, sharex=False)
    
    
    
    def annotate(data, **kws):
        ax = plt.gca()
        slope, intercept, r_value, p_value, std_err = stats.linregress(data['n_viajes'],data['tiempo_total'])
        median = data.tiempo_total.median()
        max_value_x= data.n_viajes.max()
        max_value_y= data.tiempo_total.max()
        plt.axhline(y=median, c='red')
    
        interception = linsolve([slope*x+intercept-y, median-y ], (x, y))
        (x0, y0) = next(iter(interception))
        plt.axvline(x=x0, c='green')
        #ax.map(plt.vlines, x=x0, ymin=median, ymax=max_value_y, color='black', marker='-')
        
        ax.text(0.7, .9, 'slope={:.6f}, intercept={:.6g}'.format(slope, intercept, r_value, p_value, std_err),
            transform=ax.transAxes)
        ax.text(13,median+4,'Median', color='red')
        
    ax.map_dataframe(annotate)
    ax.set(xlim=(0,15))
    ax.set_titles("{col_name}")
    #ax.fig.suptitle(hue)
    #ax.fig.subplots_adjust(top=0.9)
    ax.set_axis_labels(x_var="# de Viajes", y_var="Tiempo promedio de viaje")
    if filename:
        ax.savefig(filename)
    sns.despine()

In [ ]:
cantidad_viajes_etapa_por_sexo = viajes_gpd_etapa.groupby(['IDP_x', 'SEXO']).size()

In [ ]:
tiempo_prom_viajes_etapa_por_sexo = viajes_gpd_etapa.groupby(['IDP_x', 'SEXO']).mean('time_diff')['time_diff']

In [ ]:
cantidad_viajes_etapa_por_sexo = pd.DataFrame(cantidad_viajes_etapa_por_sexo)

In [ ]:
tiempo_prom_viajes_etapa_por_sexo = pd.DataFrame(tiempo_prom_viajes_etapa_por_sexo)

In [ ]:
n_viajes = pd.merge(cantidad_viajes_etapa_por_sexo, tiempo_prom_viajes_etapa_por_sexo, left_index=True, right_index=True)

In [ ]:
n_viajes.columns = ['n_viajes', 'tiempo_total']

In [ ]:
n_viajes['PONDERADOR_CALIBRADO'] = 1

In [ ]:
n_viajes.reset_index(inplace = True)

In [ ]:
x, y = symbols('x y')
plot_lmplot(n_viajes)

In [ ]:
groups = generate_groups(n_viajes)

In [ ]:
print(calculate_indicators(groups,'PONDERADOR_CALIBRADO'))

In [ ]:
plot_lmplot(n_viajes, col="SEXO", hue="SEXO", col_wrap=1)

In [ ]:
subgroups = n_viajes.SEXO.unique()
for element in subgroups:
    print(element)
    groups = generate_groups(n_viajes[n_viajes.SEXO==element])
    print(calculate_indicators(groups,'PONDERADOR_CALIBRADO'))